<a href="https://colab.research.google.com/github/shahaansshah/3253-083_Group9/blob/main/3253_083_Group_9_Data_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Ontario Housing Data Preprocessing

This is a rough template to start off with. Add or edit sections & code as needed.

In [33]:
import numpy as np
import pandas as pd
import json
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

### Add Proximity to lake feature to housing data?

### Import Ontario Housing data

In [34]:
# Assuming you have the JSON data in a file named 'data.json' in your Google Drive
# Mount your Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [35]:
# Specify the path to your JSON file
file_path = '/content/drive/MyDrive/Colab Notebooks/ontario_housing_cleaned.json'

# Load the JSON data from the file line by line
records = []
with open(file_path, 'r') as f:
    for line in f:
        try:
            data = json.loads(line) # Load each line as a separate JSON object
            record = data.get('data') # Use .get() to handle potential missing 'data' key
            if record is not None: # Check if 'data' exists and is not None
                features = {}
                for feature_group in record.get('features', []): # Handle cases where 'features' might be missing
                    for feature_category in feature_group.get('value', []):
                        for feature in feature_category.get('value', []):
                            features[f"{feature_group['name']}_{feature_category['name']}_{feature['name']}"] = feature['value']

                record.update(features)  # Add the extracted features to the record
                record.pop('features', None)  # Remove the original nested features structure if it exists
                records.append(record)
        except json.JSONDecodeError as e:
            print(f"Skipping invalid JSON line: {line}") # Log any lines that fail to parse

# Create the DataFrame
df = pd.DataFrame(records)

# Save the DataFrame to a CSV file
#df.to_csv('/content/drive/MyDrive/Colab Notebooks/3253/ontario_housing_cleaned.csv', index=False)


# Display the DataFrame
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3190 entries, 0 to 3189
Columns: 200 entries, address to pricePerAcreIntRaw
dtypes: bool(10), float64(17), int64(12), object(161)
memory usage: 4.7+ MB


,address,subPremise,fullAddress,division,city,cityCode,closePrice,closeDate,daysOnMovoto,daysOnMovotoRaw,description,id,tnImgPath,imagesURL,photos,latitude,listDate,listingCoAgent,listingAgent,listingAgentLicense,listingOfficePhone,listPrice,longitude,lotSize,lotSizeRaw,sqftTotal,sqftTotalRaw,mlsDbNumber,mlsSysid,mls,mlsNumber,neighborhoodName,neighborhoodN,numBathrooms,numBathroomsRaw,numBedrooms,numBedroomsRaw,openHouses,officeColistName,officeListName,officeListPhone,parking,photoCount,priceRaw,price,priceSeo,propertyType,propertyTypeName,propertyTypeValue,propertyTypeDisplayName,state,status,yearBuilt,yearBuiltRaw,zipCode,pageUrlWithoutDomain,houseRealStatus,listingOfficeDescription,totalMonthlyFee,hoafee,hoafeeRaw,dppInactiveOnActive,dppInactive,priceChanged,priceChangedDate,updatedTime,hiddenByComplianceRule,dateHidden,propertyId,neighborhoodNGeoId,visibility,permitAvm,modificationTimestamp,soldDate,createdAt,propertyDateHidden,imageDownloaderStatus,onMarketDateTime,priceChangeAmount,thumbnail,photoCount1,garage,virtualTourLink,fsa,isVOWListing,isHotHome,dppurl,isSold,listingByMovoto,isPriceReduced,label,labelclass,labelDisplayName,listingPriceFormat,addressRaw,address2,lotSizeUnit,sqftTotalUnit,comparableHomes,listDateLLFormat,listDateFormat,listDateUTC,pricePerSqft,pricePerSqftRaw,pricePerSqftIntRaw,isFavorite,petiteImagePath,propertyTypeNameUrl,Amenities Utilities_Heating Cooling_Cooling,Amenities Utilities_Heating Cooling_Heat Source,Amenities Utilities_Heating Cooling_Heat Type,Amenities Utilities_Other_Has Pool (YN),Amenities Utilities_Other_Pets Allowed (YN),Amenities Utilities_Utility_Sewer Septic,Amenities Utilities_Utility_Utilities,Exterior_Building_# Total Stories,Exterior_Building_Building Amenities,Exterior_Building_Construction Materials,Exterior_Building_Foundation,Exterior_Building_Roof,Exterior_Other_Exterior Features,Exterior_Other_Fencing,Exterior_Other_Other Structures,Exterior_Parking_# Garage Spaces,Exterior_Parking_# Parking Spaces,Exterior_Parking_Drive,Exterior_Parking_Garage Features,Exterior_Parking_Has Basement (YN),Exterior_Parking_Has Garage (YN),Exterior_Parking_Parking Desc,Exterior_Parking_Parking Spot #,Interior_Bathrooms_# Full Baths,Interior_Bathrooms_# Half Baths,Interior_Bathrooms_# Three-Quarter Baths,Interior_Bathrooms_# Total Bathrooms,Interior_Bedrooms_# of Above Grade Bedrooms,Interior_Bedrooms_# of Below Grade Bedrooms,Interior_Bedrooms_# of Rooms,Interior_Bedrooms_Family Room Available,Interior_Flooring_Flooring,Interior_Interior_Appliances,Interior_Interior_Has Fireplace (YN),Interior_Interior_Laundry Information,Interior_More rooms_# of Kitchens,Interior_Other_Interior Features,Location_Community_Community,Location_Community_Community Features,Location_Community_County,Location_Location features_Area,Location_Location features_Subdivision,Location_Location features_View,Location_Location features_Water Body Name,Location_Location features_Water Body Type,Location_Location features_Water Source,Location_Location features_Zoning Description,Location_Other_Directions,Location_Schools_Elementary School,Location_Schools_High School,Location_Schools_Middle School,Location_Schools_School District,Lot Land Details_Lot Information_Exposure,Lot Land Details_Lot Information_FarmAgriculture,Lot Land Details_Lot Information_Lot Desc,Lot Land Details_Lot Information_Lot Dimensions,Lot Land Details_Lot Information_Lot Size Units,Lot Land Details_Lot Information_Water Features,Lot Land Details_Lot Information_Water Frontage,Overview_Lot_Approx Lot Size (Range),Overview_Lot_Lot Size (Acres),Overview_Other_Approx Age,Overview_Other_Is Gated Community (YN),Overview_Other_Is Horse Property (YN),Overview_Other_Maintenance Fee,Overview_Other_New Construction (YN),Overview_Other_Year Built,Overview_Other_HOA,Overview_Property_Approx Square Feet (Range),Overview_Property_Basement Information,Overview_Property_Building Size,Overview_Property_MLS #,Overview_Property_Property Sub Type,Overview_Proper

### Handle Missing or Mostly Missing Data

In [36]:
# look up which fields contain a lot of missing data
missing_values = [col for col in df.columns if df[col].isnull().any()]
print('Number of columns with missing values:', len(missing_values), '\n')

for col in missing_values:
  print(col, round(df[col].isnull().mean(), 3)*100, '% missing values')

Number of columns with missing values: 35 

cityCode 13.600000000000001 % missing values
closePrice 100.0 % missing values
closeDate 100.0 % missing values
listDate 38.5 % missing values
listingCoAgent 0.1 % missing values
listingAgent 0.1 % missing values
lotSizeRaw 95.19999999999999 % missing values
sqftTotalRaw 3.8 % missing values
openHouses 95.0 % missing values
parking 0.5 % missing values
yearBuiltRaw 87.4 % missing values
priceChanged 97.2 % missing values
dateHidden 100.0 % missing values
neighborhoodNGeoId 7.7 % missing values
soldDate 100.0 % missing values
propertyDateHidden 100.0 % missing values
priceChangeAmount 97.39999999999999 % missing values
thumbnail 1.7999999999999998 % missing values
photoCount1 1.7999999999999998 % missing values
garage 49.5 % missing values
virtualTourLink 74.3 % missing values
fsa 0.0 % missing values
petiteImagePath 1.7999999999999998 % missing values
propertyTypeNameUrl 5.3 % missing values
virtualLink 71.2 % missing values
is3DTour 74.3 % m

In [37]:
# get columns which are >50% empty
too_many_missing = [col for col in df.columns if df[col].isnull().mean() > 0.5]
print('Number of columns with more than 50% missing values:', len(too_many_missing), '\n')

# drop those columns
try:
  df = df.drop(columns=too_many_missing)
except Exception:
  pass

Number of columns with more than 50% missing values: 22 



In [38]:
# rerun missing_values
missing_values = [col for col in df.columns if df[col].isnull().any()]
print('Number of columns with missing values:', len(missing_values), '\n')

for col in missing_values:
  print(col, round(df[col].isnull().mean(), 3)*100, '% missing values')

Number of columns with missing values: 13 

cityCode 13.600000000000001 % missing values
listDate 38.5 % missing values
listingCoAgent 0.1 % missing values
listingAgent 0.1 % missing values
sqftTotalRaw 3.8 % missing values
parking 0.5 % missing values
neighborhoodNGeoId 7.7 % missing values
thumbnail 1.7999999999999998 % missing values
photoCount1 1.7999999999999998 % missing values
garage 49.5 % missing values
fsa 0.0 % missing values
petiteImagePath 1.7999999999999998 % missing values
propertyTypeNameUrl 5.3 % missing values


### Drop other unneccessary features

For future: possible signals might include how long listing was on the site for, who were the agents associated with the sale, and others theories that might pop up based on visually looking through the features.

In [39]:
pd.options.display.max_columns = None
df

,address,subPremise,fullAddress,division,city,cityCode,daysOnMovoto,daysOnMovotoRaw,description,id,tnImgPath,imagesURL,photos,latitude,listDate,listingCoAgent,listingAgent,listingAgentLicense,listingOfficePhone,listPrice,longitude,lotSize,sqftTotal,sqftTotalRaw,mlsDbNumber,mlsSysid,mls,mlsNumber,neighborhoodName,neighborhoodN,numBathrooms,numBathroomsRaw,numBedrooms,numBedroomsRaw,officeColistName,officeListName,officeListPhone,parking,photoCount,priceRaw,price,priceSeo,propertyType,propertyTypeName,propertyTypeValue,propertyTypeDisplayName,state,status,yearBuilt,zipCode,pageUrlWithoutDomain,houseRealStatus,listingOfficeDescription,totalMonthlyFee,hoafee,hoafeeRaw,dppInactiveOnActive,dppInactive,priceChangedDate,updatedTime,hiddenByComplianceRule,propertyId,neighborhoodNGeoId,visibility,permitAvm,modificationTimestamp,createdAt,imageDownloaderStatus,onMarketDateTime,thumbnail,photoCount1,garage,fsa,isVOWListing,isHotHome,dppurl,isSold,listingByMovoto,isPriceReduced,label,labelclass,labelDisplayName,listingPriceFormat,addressRaw,address2,lotSizeUnit,sqftTotalUnit,comparableHomes,listDateLLFormat,listDateFormat,listDateUTC,pricePerSqft,pricePerSqftRaw,pricePerSqftIntRaw,isFavorite,petiteImagePath,propertyTypeNameUrl,Amenities Utilities_Heating Cooling_Cooling,Amenities Utilities_Heating Cooling_Heat Source,Amenities Utilities_Heating Cooling_Heat Type,Amenities Utilities_Other_Has Pool (YN),Amenities Utilities_Other_Pets Allowed (YN),Amenities Utilities_Utility_Sewer Septic,Amenities Utilities_Utility_Utilities,Exterior_Building_# Total Stories,Exterior_Building_Building Amenities,Exterior_Building_Construction Materials,Exterior_Building_Foundation,Exterior_Building_Roof,Exterior_Other_Exterior Features,Exterior_Other_Fencing,Exterior_Other_Other Structures,Exterior_Parking_# Garage Spaces,Exterior_Parking_# Parking Spaces,Exterior_Parking_Drive,Exterior_Parking_Garage Features,Exterior_Parking_Has Basement (YN),Exterior_Parking_Has Garage (YN),Exterior_Parking_Parking Desc,Exterior_Parking_Parking Spot #,Interior_Bathrooms_# Full Baths,Interior_Bathrooms_# Half Baths,Interior_Bathrooms_# Three-Quarter Baths,Interior_Bathrooms_# Total Bathrooms,Interior_Bedrooms_# of Above Grade Bedrooms,Interior_Bedrooms_# of Below Grade Bedrooms,Interior_Bedrooms_# of Rooms,Interior_Bedrooms_Family Room Available,Interior_Flooring_Flooring,Interior_Interior_Appliances,Interior_Interior_Has Fireplace (YN),Interior_Interior_Laundry Information,Interior_More rooms_# of Kitchens,Interior_Other_Interior Features,Location_Community_Community,Location_Community_Community Features,Location_Community_County,Location_Location features_Area,Location_Location features_Subdivision,Location_Location features_View,Location_Location features_Water Body Name,Location_Location features_Water Body Type,Location_Location features_Water Source,Location_Location features_Zoning Description,Location_Other_Directions,Location_Schools_Elementary School,Location_Schools_High School,Location_Schools_Middle School,Location_Schools_School District,Lot Land Details_Lot Information_Exposure,Lot Land Details_Lot Information_FarmAgriculture,Lot Land Details_Lot Information_Lot Desc,Lot Land Details_Lot Information_Lot Dimensions,Lot Land Details_Lot Information_Lot Size Units,Lot Land Details_Lot Information_Water Features,Lot Land Details_Lot Information_Water Frontage,Overview_Lot_Approx Lot Size (Range),Overview_Lot_Lot Size (Acres),Overview_Other_Approx Age,Overview_Other_Is Gated Community (YN),Overview_Other_Is Horse Property (YN),Overview_Other_Maintenance Fee,Overview_Other_New Construction (YN),Overview_Other_Year Built,Overview_Other_HOA,Overview_Property_Approx Square Feet (Range),Overview_Property_Basement Information,Overview_Property_Building Size,Overview_Property_MLS #,Overview_Property_Property Sub Type,Overview_Property_Status,Overview_Property_Storage Unit (Locker),Overview_Property_Virtual Tour,Overview_Taxes_Tax Year,Overview_Taxes_Taxes,Rooms_Rooms Info

In [40]:
# notice that 'garage' is almost half empty, but 'parking' is almost full - check if they hold the same kind of data (they do)
# print(df['garage'].value_counts(), '\n', df['parking'].value_counts()

# drop 'garage'
# try:
#   df = df.drop(columns=['garage'])
# except Exception:
#   pass

In [41]:
# Drop the specified columns
# These features are either irrelevant for predicting price, redundant, or contain mostly null values.
columns_to_drop = ['tnImgPath', 'imagesURL', 'photos', 'id', 'listingCoAgent', 'listingAgent',
                   'listingAgentLicense', 'listingOfficePhone',
                   'mlsDbNumber', 'mlsSysid', 'mls', 'mlsNumber', 'openHouses',
                   'officeColistName', 'officeListName', 'officeListPhone', 'photoCount', 'price', 'priceSeo', 'propertyTypeValue', 'propertyTypeDisplayName'
                   'state', 'status', 'pageUrlWithoutDomain', 'houseRealStatus', 'listingOfficeDescription', 'hoafee', 'hoafeeRaw', 'dppInactiveOnActive',
                   'dppInactive', 'priceChanged', 'priceChangedDate', 'updatedTime', 'hiddenByComplianceRule', 'dateHidden', 'propertyId', 'visibility',
                   'permitAvm', 'modificationTimestamp', 'createdAt', 'propertyDateHidden', 'imageDownloaderStatus', 'onMarketDateTime', 'priceChangeAmount', 'thumbnail',
                   'photoCount1', 'virtualTourLink', 'fsa', 'dppurl', 'listingByMovoto', 'labelDisplayName', 'listingPriceFormat', 'comparableHomes',
                   'listDateLLFormat', 'listDateLLFormat', 'listDateFormat', 'listDateUTC', 'pricePerSqft', 'pricePerSqftRaw', 'pricePerSqftIntRaw', 'isFavorite',
                   'petiteImagePath', 'propertyTypeNameUrl', 'closePrice', 'closeDate', 'daysOnMovoto', 'cityCode', 'listDate',
                   'state', 'soldDate', 'isHotHome', 'isSold', 'isPriceReduced', 'label', 'labelclass', 'Amenities  Utilities_Other_Pets Allowed (YN)',
                   'Amenities  Utilities_Utility_Utilities', 'Exterior_Building_# Total Stories', 'Exterior_Building_Building Amenities',
                   'Exterior_Building_Building Amenities', 'Exterior_Other_Exterior Features', 'Exterior_Other_Fencing', 'Exterior_Building_Foundation',
                   'Exterior_Parking_# Garage Spaces', 'Exterior_Parking_Drive', 'Exterior_Parking_Garage Features', 'Exterior_Parking_Has Basement (YN)',
                   'Exterior_Parking_Has Garage (YN)', 'Exterior_Parking_Parking Desc', 'Exterior_Parking_Parking Spot #',
                   'Interior_Bathrooms_# Full Baths', 'Interior_Bathrooms_# Half Baths', 'Interior_Bathrooms_# Three-Quarter', 'Interior_Bathrooms_# Total Bathrooms',
                   'Interior_Bedrooms_Family Room Available', 'Interior_Flooring_Flooring', 'Interior_Interior_Appliances', 'Interior_Interior_Has Fireplace (YN)',
                   'Interior_Interior_Laundry Information', 'Interior_Other_Interior Features', 'Location_Community_Community', 'Location_Community_Community Features',
                   'Location_Community_County', 'Location_Location features_Area', 'Location_Location features_Subdivision', 'Location_Location features_View',
                   'Location_Location features_Water Body Name', 'Location_Location features_Water Body Type', 'Location_Location features_Water Source',
                   'Location_Location features_Zoning Description', 'Location_Other_Directions', 'Location_Schools_Elementary School', 'Location_Schools_High School',
                   'Location_Schools_Middle School', 'Location_Schools_School District', 'Lot Land Details_Lot Information_Exposure',
                   'Lot Land Details_Lot Information_FarmAgriculture', 'Lot Land Details_Lot Information_Lot Desc', 'Lot Land Details_Lot Information_Lot Size Units',
                   'Lot Land Details_Lot Information_Water Features', 'Lot Land Details_Lot Information_Water Frontage', 'Overview_Lot_Approx Lot Size (Range)',
                   'Overview_Other_Approx Age', 'Overview_Other_Is Gated Community (YN)', 'Overview_Other_Is Horse Property (YN)', 'Overview_Other_New Construction (YN)',
                   'Overview_Other_Year Built', 'Overview_Other_HOA', 'Overview_Property_Approx Square Feet (Range)', 'Overview_Property_MLS #',
                   'Overview_Property_Status', 'Overview_Property_Storage Unit (Locker)', 'Overview_Property_Virtual Tour', 'Overview_Taxes_Tax Year', 'Overview_Taxes_Taxes',
                   'Rooms_Rooms Information_Movotorooms', 'SOA_HOUSEKEEPING_ATTRS_LISTING_SOURCE_URL_Listing Source URL',
                   'SOA_HOUSEKEEPING_ATTRS_LISTING_TYPE_Listing Type Identifier', 'virtualLink', 'is3DTour', 'isPriceUp', 'priceChange', 'priceChangeFriendlyPrice',
                   'lastListPriceRaw', 'lastListPrice', 'pricePerAcre', 'pricePerAcreRaw', 'pricePerAcreIntRaw', 'subPremise', 'fullAddress', 'division', 'daysOnMovotoRaw',
                   'description', 'lotSizeRaw', 'sqftTotalRaw', 'neighborhoodN', 'numBathroomsRaw', 'numBedroomsRaw', 'propertyTypeName','propertyTypeDisplayName',
                   'yearBuiltRaw', 'totalMonthlyFee', 'neighborhoodNGeoId', 'isVOWListing', 'addressRaw', 'address2', 'lotSizeUnit', 'sqftTotalUnit',
                   'Amenities  Utilities_Utility_Sewer Septic', 'Amenities  Utilities_Heating  Cooling_Heat Type', 'Amenities  Utilities_Other_Has Pool (YN)',
                   'Amenities  Utilities_Utility_Utility_Sewer Septic', 'Exterior_Building_Construction Materials', 'Exterior_Building_Roof',
                   'Exterior_Other_Other Structures', 'Exterior_Parking_# Parking Spaces', 'Interior_Bathrooms_# Three-Quarter Baths',
                   'Interior_Bedrooms_# of Above Grade Bedrooms', 'Interior_Bedrooms_# of Below Grade Bedrooms', 'Interior_Bedrooms_# of Rooms',
                   'Interior_More rooms_# of Kitchens', 'Lot Land Details_Lot Information_Lot Dimensions', 'Overview_Lot_Lot Size (Acres)', 'Overview_Other_Maintenance Fee',
                   'Overview_Property_Basement Information', 'Overview_Property_Building Size ', 'Overview_Property_Property Sub Type',
                   'city', 'neighborhoodName', 'priceChangeRaw', 'address', 'zipCode']

# Use errors='ignore' to avoid errors if a column doesn't exist, drop in-place
df.drop(columns=columns_to_drop, axis=1, inplace=True, errors='ignore')


In [42]:
df

,latitude,listPrice,longitude,lotSize,sqftTotal,numBathrooms,numBedrooms,parking,priceRaw,propertyType,yearBuilt,garage,Amenities Utilities_Heating Cooling_Cooling,Amenities Utilities_Heating Cooling_Heat Source
0,43.497160,995000,-79.707940,,,2,2,2.0,995000,Single Family,...,NaN,Central air conditioning,Natural gas
1,43.479693,1190000,-79.682036,,,2,3,3.0,1190000,Single Family,...,NaN,Central air conditioning,
2,43.494279,1450000,-79.742757,,,5,5,2.0,1450000,TOWNHOUSE,...,1.0,Central Air,Gas
3,43.476904,579900,-79.732563,,600,1,1,1.0,579900,Condo,2022,1.0,Central Air,
4,43.495964,2799900,-79.719458,,"3,993",6,5,4.0,2799900,Single Family,2018,2.0,Central Air,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3185,43.387931,845900,-80.472507,,"2,147",4,4,3.0,845900,Single Family,2005,2.0,Central Air,
3186,43.443545,735425,-80.494485,,"1,154",3,2,1.0,735425,Single Family,2025,NaN,Ductless,Electric
3187,43.561960,749900,-80.257167,,,...,...,0.0,749900,Land,...,NaN,,
3188,43.424894,409900,-80.444726,,,1,2,1.0,409900,Single Family,...,NaN,,Electric


### Display correlation heatmap of remaining features that we believe are of interest
- drop redundant features
- drop useless features

### Scale Features

### Obtain training and testing data.

In [43]:
y, X = df.pop('priceRaw'), df

# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

0        995000
1       1190000
2       1450000
3        579900
4       2799900
         ...   
3185     845900
3186     735425
3187     749900
3188     409900
3189     575000
Name: priceRaw, Length: 3190, dtype: int64


,latitude,listPrice,longitude,lotSize,sqftTotal,numBathrooms,numBedrooms,parking,propertyType,yearBuilt,garage,Amenities Utilities_Heating Cooling_Cooling,Amenities Utilities_Heating Cooling_Heat Source
0,43.497160,995000,-79.707940,,,2,2,2.0,Single Family,...,NaN,Central air conditioning,Natural gas
1,43.479693,1190000,-79.682036,,,2,3,3.0,Single Family,...,NaN,Central air conditioning,
2,43.494279,1450000,-79.742757,,,5,5,2.0,TOWNHOUSE,...,1.0,Central Air,Gas
3,43.476904,579900,-79.732563,,600,1,1,1.0,Condo,2022,1.0,Central Air,
4,43.495964,2799900,-79.719458,,"3,993",6,5,4.0,Single Family,2018,2.0,Central Air,
